In [1]:
import numpy as np
import pandas as pd
from keras import layers
from keras import models
from keras import callbacks
from keras import backend as K
from keras.datasets import mnist
from matplotlib import pyplot as plt
from matplotlib import cm
from PIL import Image
import glob
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import jaccard_score,log_loss, mean_absolute_error,median_absolute_error
%matplotlib inline

Using TensorFlow backend.


In [2]:
from tqdm import tqdm
import pickle
def makefile(what,filename):
    with open(filename,"wb") as f3:
        pickle.dump(what,f3)

def readfile(filename):
    with open(filename,"rb") as f4:
        ans=pickle.load(f4)
    return ans

In [3]:
from keras.datasets import cifar10

In [4]:
from keras.datasets import cifar100

In [5]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import balanced_accuracy_score

In [6]:
images=["Lenna.png"]

In [7]:
from keras.backend import clear_session

In [8]:
import numpy as np
import random
import tensorflow.compat.v1.keras.backend as K
import tensorflow as tf

In [9]:
y2=Image.open("Lenna.png").resize([32,32])

In [10]:
np.random.seed(seed=1)

os.environ['PYTHONHASHSEED'] = '0'
random.seed(0)

session_conf = tf.compat.v1.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)
tf.random.set_seed(0)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
K.set_session(sess)
degree=[0,90,180,270]
values=[0,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4]
convo=[6]
for goal in images:
    y2=Image.open(goal).resize([32,32])
    data2=cifar10.load_data()[0][0]
    label2=cifar10.load_data()[0][1]
    label2=label2.reshape(len(label2))
    x_test=cifar10.load_data()[1][0]
    y_test=cifar10.load_data()[1][1]
    y_test=y_test.reshape(len(y_test))
    data3=np.zeros([len(data2),32,32,3])
    for n in range(len(data2)):
        data3[n]=Image.fromarray(data2[n].astype("uint8"))
    data2=data3
    
    outputs=[]
    outputs4=[]
    outputs5=[]
    for c in range(len(convo)):
        outputs2=[]
        outputs3=[]
        for e in range(5):
            ensemble=[]
            for rot in degree:
                error_record=[]
                clear_session()
                unseen_score=np.zeros([10,9])
                AUC_score=np.zeros(10)
                for i in range(10):
                    x_train=data2[label2==i]
                    y_train=np.zeros([len(x_train),32,32,3])
                    for n in range(len(x_train)):
                        y_train[n]=y2.rotate(rot)
                    #x_train=x_train/255
                    y_train=y_train
                    #x_test=data2[label2!=i]
                    #x_test=x_test/255
                    x_test=x_test.reshape(len(x_test),32,32,3)
                    y_label=np.zeros(len(y_test))
                    y_label[y_test==i]=1
                    #(x_train, train), (x_test, y_test) = mnist.load_data()
                    image_height, image_width = 32, 32
                    # バックエンドに依存したチャネルの位置を調整する
                    if K.image_data_format() == 'channels_last':
                        x_train = x_train.reshape(x_train.shape[0],
                                                  image_height, image_width,3)
                        #x_test = x_test.reshape(x_test.shape[0],image_height, image_width, 1)
                        y_train = y_train.reshape(y_train.shape[0],
                                                  32, 32,3)
                        input_shape = (image_height, image_width,3)
                    else:
                        x_train = x_train.reshape(x_train.shape[0],
                                                  1, image_height, image_width)
                        x_test = x_test.reshape(x_test.shape[0],1, image_height, image_width)
                        input_shape = (1, image_height, image_width)

                    # Min-Max Normalization (0. ~ 1. の範囲に値を収める)
                    x_train = x_train.astype('float32')
                    y_train = y_train.astype('float32')
                    x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.3, random_state=e)
                    x_valid, x_valid2, y_valid, y_valid2 = train_test_split(x_valid, y_valid, test_size=0.3, random_state=e)
                    #x_test = x_test.astype('float32')
                    #x_train = (x_train - x_train.min()) / (x_train.max() - x_train.min())
                    #x_test = (x_test - x_test.min()) / (x_test.max() - x_test.min())
                    # 畳み込み演算を用いた AutoEncoder のネットワーク (Sequential API)
                    model = models.Sequential()
                    # 28 x 28 x 1
                    model.add(layers.Conv2D(32,kernel_size=(3,3),activation='relu',padding='same',input_shape=input_shape))
                    for n in range(convo[c]):
                        model.add(layers.Conv2D(32,kernel_size=(3,3),activation='relu',padding='same'))
                    model.add(layers.Conv2D(3, kernel_size=(3, 3),activation='relu', padding='same'))
                    # 28 x 28 x 1
                    model.compile(optimizer='adam',
                                  loss='mean_absolute_error')
                    # モデルの構造を確認する
                    fit_callbacks = [
                        callbacks.EarlyStopping(monitor='val_loss',
                                                patience=5,
                                                mode='min')
                    ]
                    # モデルを学習させる
                    model.fit(x_train, y_train,
                              epochs=100,
                              batch_size=16,
                              shuffle=True,
                              validation_data=(x_valid, y_valid),callbacks=fit_callbacks)
                    result=model.predict(x_valid2)
                    result2=model.predict(x_test)
                    y_train=y_train.reshape(y_train.shape[0],1024,3)
                    result=result.reshape(result.shape[0],1024,3)
                    result2=result2.reshape(result2.shape[0],1024,3)
                    loss=np.zeros(len(result))
                    for n in range(len(result)):
                        loss[n]=mean_absolute_error(y_train[0],result[n])
                    loss_outlier=np.zeros(len(result2))
                    for n in range(len(result2)):
                        loss_outlier[n]=mean_absolute_error(y_train[0],result2[n])
                    error_record.append(loss_outlier)
                    #y_one=np.ones(len(loss))
                    #y_outlier=np.zeros(len(loss_outlier))
                    #data=np.concatenate([loss,loss_outlier])
                    #label=np.concatenate([y_one,y_outlier])
                    #plt.scatter(data,label)
                    #plt.hist([loss,loss_outlier],range=(0,0.3),stacked=False,bins=10)
                    m=0
                    AUC_score[i]=roc_auc_score(y_label,(-1)*loss_outlier)
                    for p in values:
                        y_pred=np.zeros(len(y_label))
                        threshold=int(len(loss)*p)+1
                        #print(loss[loss.argsort()[-threshold]])
                        y_pred[loss_outlier<=loss[loss.argsort()[-threshold]]]=1
                        #loss_outlier[loss_outlier>loss[loss.argsort()[-threshold]]
                        unseen_score[i][m]=balanced_accuracy_score(y_label,y_pred)
                        m+=1
                #print(unseen_score)
                #outputs.append(unseen_score)
                outputs2.append(AUC_score)
                outputs3.append(unseen_score)
                ensemble.append(error_record)
            outputs5.append(ensemble)
            outputs.append(outputs2)
            outputs4.append(outputs3)
    #makefile(outputs,"result_ocitn/cifar10_"+str(goal)+".pkl")

Train on 3500 samples, validate on 1050 samples
Epoch 1/100
3500/3500 [==============================] - 5s 1ms/step - loss: 42.4447 - val_loss: 37.3438
Epoch 2/100
3500/3500 [==============================] - 2s 713us/step - loss: 35.7112 - val_loss: 36.7981
Epoch 3/100
3500/3500 [==============================] - 3s 782us/step - loss: 33.1963 - val_loss: 32.7848
Epoch 4/100
3500/3500 [==============================] - 2s 687us/step - loss: 31.5250 - val_loss: 30.0166
Epoch 5/100
3500/3500 [==============================] - 2s 625us/step - loss: 29.9285 - val_loss: 27.9064
Epoch 6/100
3500/3500 [==============================] - 2s 593us/step - loss: 25.9135 - val_loss: 23.5356
Epoch 7/100
3500/3500 [==============================] - 2s 706us/step - loss: 22.6266 - val_loss: 22.1481
Epoch 8/100
3500/3500 [==============================] - 2s 609us/step - loss: 21.7953 - val_loss: 21.7224
Epoch 9/100
3500/3500 [==============================] - 2s 669us/step - loss: 21.4533 - val_loss:

3500/3500 [==============================] - 2s 579us/step - loss: 19.8593 - val_loss: 20.0252
Epoch 37/100
3500/3500 [==============================] - 2s 560us/step - loss: 19.8470 - val_loss: 20.1786
Epoch 38/100
3500/3500 [==============================] - 2s 567us/step - loss: 19.9225 - val_loss: 20.0002
Train on 3500 samples, validate on 1050 samples
Epoch 1/100
3500/3500 [==============================] - 2s 669us/step - loss: 45.3122 - val_loss: 40.4504
Epoch 2/100
3500/3500 [==============================] - 2s 614us/step - loss: 37.9817 - val_loss: 36.1686
Epoch 3/100
3500/3500 [==============================] - 2s 558us/step - loss: 35.0028 - val_loss: 32.6035
Epoch 4/100
3500/3500 [==============================] - 2s 575us/step - loss: 33.2797 - val_loss: 30.4047
Epoch 5/100
3500/3500 [==============================] - 2s 556us/step - loss: 27.2544 - val_loss: 24.5077
Epoch 6/100
3500/3500 [==============================] - 2s 560us/step - loss: 23.5129 - val_loss: 22.9024

3500/3500 [==============================] - 2s 554us/step - loss: 20.1067 - val_loss: 20.0702
Epoch 26/100
3500/3500 [==============================] - 2s 555us/step - loss: 20.1410 - val_loss: 20.0419
Epoch 27/100
3500/3500 [==============================] - 2s 556us/step - loss: 20.0805 - val_loss: 19.9801
Epoch 28/100
3500/3500 [==============================] - 2s 562us/step - loss: 20.0255 - val_loss: 20.0921
Epoch 29/100
3500/3500 [==============================] - 2s 554us/step - loss: 20.0293 - val_loss: 19.9530
Epoch 30/100
3500/3500 [==============================] - 2s 555us/step - loss: 19.9660 - val_loss: 20.1340
Epoch 31/100
3500/3500 [==============================] - 2s 551us/step - loss: 19.9309 - val_loss: 19.9996
Epoch 32/100
3500/3500 [==============================] - 2s 558us/step - loss: 19.9189 - val_loss: 19.9531
Epoch 33/100
3500/3500 [==============================] - 2s 556us/step - loss: 19.8536 - val_loss: 20.0634
Epoch 34/100
3500/3500 [=================

3500/3500 [==============================] - 2s 662us/step - loss: 19.1955 - val_loss: 19.3831
Epoch 45/100
3500/3500 [==============================] - 2s 617us/step - loss: 19.1161 - val_loss: 19.4750
Epoch 46/100
3500/3500 [==============================] - 2s 558us/step - loss: 19.0940 - val_loss: 19.3917
Epoch 47/100
3500/3500 [==============================] - 2s 550us/step - loss: 19.0640 - val_loss: 19.4684
Train on 3500 samples, validate on 1050 samples
Epoch 1/100
3500/3500 [==============================] - 2s 682us/step - loss: 41.6658 - val_loss: 36.7891
Epoch 2/100
3500/3500 [==============================] - 2s 573us/step - loss: 33.6349 - val_loss: 33.0373
Epoch 3/100
3500/3500 [==============================] - 2s 595us/step - loss: 31.3566 - val_loss: 30.9502
Epoch 4/100
3500/3500 [==============================] - 2s 578us/step - loss: 30.2162 - val_loss: 29.1582
Epoch 5/100
3500/3500 [==============================] - 2s 571us/step - loss: 28.9300 - val_loss: 27.667

3500/3500 [==============================] - 2s 562us/step - loss: 18.0249 - val_loss: 18.6348
Epoch 49/100
3500/3500 [==============================] - 2s 558us/step - loss: 18.0527 - val_loss: 18.7634
Epoch 50/100
3500/3500 [==============================] - 2s 562us/step - loss: 18.0443 - val_loss: 18.6665
Epoch 51/100
3500/3500 [==============================] - 2s 572us/step - loss: 17.9298 - val_loss: 18.7473
Epoch 52/100
3500/3500 [==============================] - 2s 562us/step - loss: 17.8753 - val_loss: 18.6801
Epoch 53/100
3500/3500 [==============================] - 2s 560us/step - loss: 17.9224 - val_loss: 18.7493
Train on 3500 samples, validate on 1050 samples
Epoch 1/100
3500/3500 [==============================] - 2s 678us/step - loss: 45.0611 - val_loss: 38.9021
Epoch 2/100
3500/3500 [==============================] - 2s 570us/step - loss: 37.7724 - val_loss: 36.4241
Epoch 3/100
3500/3500 [==============================] - 2s 577us/step - loss: 35.7093 - val_loss: 33.8

...

3500/3500 [==============================] - 2s 543us/step - loss: 21.6665 - val_loss: 22.0669
Epoch 9/100
3500/3500 [==============================] - 2s 551us/step - loss: 21.4243 - val_loss: 21.4560
Epoch 10/100
3500/3500 [==============================] - 2s 549us/step - loss: 21.1721 - val_loss: 21.0480
Epoch 11/100
3500/3500 [==============================] - 2s 591us/step - loss: 20.9864 - val_loss: 21.1756
Epoch 12/100
3500/3500 [==============================] - 2s 582us/step - loss: 20.8510 - val_loss: 20.9777
Epoch 13/100
3500/3500 [==============================] - 2s 572us/step - loss: 20.7117 - val_loss: 20.7553
Epoch 14/100
3500/3500 [==============================] - 2s 568us/step - loss: 20.6034 - val_loss: 20.6212
Epoch 15/100
3500/3500 [==============================] - 2s 573us/step - loss: 20.5197 - val_loss: 20.5542
Epoch 16/100
3500/3500 [==============================] - 2s 572us/step - loss: 20.4218 - val_loss: 20.5891
Epoch 17/100
3500/3500 [==================

3500/3500 [==============================] - 2s 562us/step - loss: 21.5025 - val_loss: 21.4007
Epoch 9/100
3500/3500 [==============================] - 2s 561us/step - loss: 21.1422 - val_loss: 21.4886
Epoch 10/100
3500/3500 [==============================] - 2s 561us/step - loss: 20.9530 - val_loss: 20.9515
Epoch 11/100
3500/3500 [==============================] - 2s 562us/step - loss: 20.8053 - val_loss: 20.7648
Epoch 12/100
3500/3500 [==============================] - 2s 566us/step - loss: 20.6424 - val_loss: 20.8344
Epoch 13/100
3500/3500 [==============================] - 2s 564us/step - loss: 20.5163 - val_loss: 20.5974
Epoch 14/100
3500/3500 [==============================] - 2s 562us/step - loss: 20.4481 - val_loss: 20.3918
Epoch 15/100
3500/3500 [==============================] - 2s 566us/step - loss: 20.3094 - val_loss: 20.4909
Epoch 16/100
3500/3500 [==============================] - 2s 561us/step - loss: 20.2361 - val_loss: 20.1939
Epoch 17/100
3500/3500 [==================

3500/3500 [==============================] - 2s 494us/step - loss: 20.3483 - val_loss: 20.2280
Epoch 22/100
3500/3500 [==============================] - 1s 384us/step - loss: 20.2504 - val_loss: 20.3382
Epoch 23/100
3500/3500 [==============================] - 2s 541us/step - loss: 20.2567 - val_loss: 20.4410
Epoch 24/100
3500/3500 [==============================] - 2s 601us/step - loss: 20.1510 - val_loss: 20.1915
Epoch 25/100
3500/3500 [==============================] - 2s 604us/step - loss: 20.1604 - val_loss: 20.1836
Epoch 26/100
3500/3500 [==============================] - 2s 596us/step - loss: 20.1059 - val_loss: 20.1151
Epoch 27/100
3500/3500 [==============================] - 2s 610us/step - loss: 20.0935 - val_loss: 20.3948
Epoch 28/100
3500/3500 [==============================] - 2s 605us/step - loss: 20.0768 - val_loss: 20.8914
Epoch 29/100
3500/3500 [==============================] - 2s 597us/step - loss: 20.0807 - val_loss: 20.2323
Epoch 30/100
3500/3500 [=================

3500/3500 [==============================] - 2s 580us/step - loss: 19.4619 - val_loss: 19.7908
Epoch 46/100
3500/3500 [==============================] - 2s 561us/step - loss: 19.4644 - val_loss: 19.7419
Epoch 47/100
3500/3500 [==============================] - 2s 601us/step - loss: 19.6759 - val_loss: 19.7165
Epoch 48/100
3500/3500 [==============================] - 2s 597us/step - loss: 19.4344 - val_loss: 19.7855
Epoch 49/100
3500/3500 [==============================] - 2s 602us/step - loss: 19.3365 - val_loss: 19.7900
Epoch 50/100
3500/3500 [==============================] - 2s 598us/step - loss: 19.3765 - val_loss: 19.7796
Epoch 51/100
3500/3500 [==============================] - 2s 601us/step - loss: 19.3310 - val_loss: 19.8122
Epoch 52/100
3500/3500 [==============================] - 2s 604us/step - loss: 19.3088 - val_loss: 19.8097
Train on 3500 samples, validate on 1050 samples
Epoch 1/100
3500/3500 [==============================] - 2s 699us/step - loss: 45.6761 - val_loss: 43

3500/3500 [==============================] - 2s 560us/step - loss: 19.0447 - val_loss: 19.3252
Epoch 50/100
3500/3500 [==============================] - 2s 561us/step - loss: 19.0499 - val_loss: 19.3861
Epoch 51/100
3500/3500 [==============================] - 2s 554us/step - loss: 18.9869 - val_loss: 19.2761
Epoch 52/100
3500/3500 [==============================] - 2s 563us/step - loss: 18.9883 - val_loss: 19.2445
Epoch 53/100
3500/3500 [==============================] - 2s 558us/step - loss: 18.9178 - val_loss: 19.3381
Epoch 54/100
3500/3500 [==============================] - 2s 558us/step - loss: 18.9225 - val_loss: 19.2522
Epoch 55/100
3500/3500 [==============================] - 2s 555us/step - loss: 18.8830 - val_loss: 19.2977
Epoch 56/100
3500/3500 [==============================] - 2s 550us/step - loss: 18.8323 - val_loss: 19.2225
Epoch 57/100
3500/3500 [==============================] - 2s 555us/step - loss: 18.8103 - val_loss: 19.2856
Epoch 58/100
3500/3500 [=================

In [12]:
outputs6=[]
for ensemble in outputs5:
    CEtest2=None
    value = 0
    for scores in ensemble:
        if type(CEtest2)==type(None):
            CEtest2=np.array(scores)
        else:
            print("OK")
            CEtest2=CEtest2+np.array(scores)
    AUC_ensemble=np.zeros(10)
    for i in range(10):
        y_label=np.zeros(len(y_test))
        y_label[y_test==i]=1
        AUC_ensemble[i]=roc_auc_score(y_label,(-1)*CEtest2[i])
    outputs6.append(AUC_ensemble)

OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK
OK


In [13]:
np.array(outputs6).mean(axis=0)

array([0.79602173, 0.8989173 , 0.70439579, 0.61948519, 0.72614533,
       0.74644768, 0.81420831, 0.82610296, 0.85028136, 0.86404527])

In [14]:
np.array(outputs6).std(axis=0)

array([0.00407343, 0.00203884, 0.0084158 , 0.01112961, 0.00734447,
       0.01307284, 0.00744207, 0.00693525, 0.00493365, 0.00192705])

In [16]:
AUC_score2=[]
for ensemble in outputs5:
    AUC_score=[]
    for scores in ensemble:
        AUC=np.zeros(10)
        for i in range(10):
            y_label=np.zeros(len(y_test))
            y_label[y_test==i]=1
            AUC[i]=roc_auc_score(y_label,(-1)*scores[i])
        AUC_score.append(AUC)
    AUC_score2.append(AUC_score)

In [21]:
outputs7=[]
outputs7.append(np.array(AUC_score2).mean(axis=0))
outputs7.append(np.array(AUC_score2).std(axis=0))
outputs7.append(np.array(outputs6).mean(axis=0))
outputs7.append(np.array(outputs6).std(axis=0))

In [22]:
outputs7

[array([[0.7726878 , 0.8737425 , 0.67841177, 0.59965668, 0.70868912,
         0.71902713, 0.76676372, 0.79597261, 0.8203519 , 0.83908724],
        [0.75102288, 0.84830759, 0.67480653, 0.57766873, 0.6885307 ,
         0.68992127, 0.73687826, 0.78050436, 0.79721544, 0.81117511],
        [0.77998154, 0.85984271, 0.66151013, 0.55986129, 0.68720491,
         0.68222457, 0.75256234, 0.77809603, 0.83178633, 0.82814534],
        [0.7613349 , 0.86077324, 0.67813643, 0.60717801, 0.70125844,
         0.70695848, 0.765069  , 0.8029526 , 0.80501179, 0.81353789]]),
 array([[0.01344732, 0.00960487, 0.00922282, 0.01208025, 0.01247462,
         0.02986395, 0.00479556, 0.01063049, 0.00752526, 0.00422531],
        [0.0059232 , 0.0049563 , 0.02117237, 0.02073257, 0.0062178 ,
         0.01396073, 0.02314738, 0.01001796, 0.00909807, 0.00946674],
        [0.00553185, 0.01585691, 0.01622124, 0.0218339 , 0.00838158,
         0.00511823, 0.00614316, 0.02838355, 0.00742722, 0.01326604],
        [0.01008402, 0.00

In [23]:
makefile(outputs7,"ITNensemble/cifar10.pkl")